In [1]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [2]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [3]:
NL1 = 2
NL2 = 2
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [4]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [5]:
## Let us compute the thermal density matrix
beta1 = 1  #left baths
beta2 = 0.9 #right baths

rho_th = (-beta1*H_S).expm()/((-beta1*H_S).expm()).tr() #left thermal density matrix
print(rho_th)

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[3.80777093e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.03175783e-03 3.29634248e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.29634248e-06 1.02846676e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.61523826e-02
  1.79974202e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.79974202e-04
  5.63320690e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.53616443e-01]]


In [6]:
## Now we set parameters for the  non-eq setup

beta_list_left = np.linspace(beta1,beta1,NL1)
beta_list_right = np.linspace(beta2,beta2,NL2)

mu_list_left = np.linspace(-0.5,-0.5,NL1)
mu_list_right = np.linspace(-0.5,-0.5,NL2)

gamma_list_left = np.linspace(1,1,NL1)  #coupling strengths to left baths
gamma_list_right = np.linspace(1,1,NL2)  #coupling strengths to right baths

tb = 0.01

In [7]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*C(k,y,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*D(y,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

            for l in range(NL2):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*C(k,y,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*D(y,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [8]:
print(number)  #should be 2^N

64


In [9]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[np.complex128(0.1536164432357538+0j), np.complex128(0.05651233129231378+0j), np.complex128(0.056463896272977196+0j), np.complex128(0.05633178086692867+0j), np.complex128(0.05615180727947186+0j), np.complex128(0.055972408687012254+0j), np.complex128(0.055841443273902114+0j), np.complex128(0.020789724855111807+0j), np.complex128(0.020771906607265127+0j), np.complex128(0.02075165768501958+0j), np.complex128(0.020723304065517863+0j), np.complex128(0.020710947145894693+0j), np.complex128(0.020674601227246826+0j), np.complex128(0.02065709548273863+0j), np.complex128(0.020629746935722004+0j), np.complex128(0.020591098428797645+0j), np.complex128(0.020560107729941855+0j), np.complex128(0.020542918945809906+0j), np.complex128(0.020509188788367952+0j), np.complex128(0.020490634791516974+0j), np.complex128(0.02043729694990147+0j), np.complex128(0.020361504768194885+0j), np.complex128(0.007648112361806576+0j), np.complex128(0.007641557394746085+0j), np.complex128(0.007634108232546068+0j), np.comp

In [10]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j
 -0.-0.j]


In [11]:
print(A)

[[ 1.26004965e+000+0.00000000e+000j -8.70313963e-001+0.00000000e+000j
  -1.24789653e+000+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.31309218e-125+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-2.10280741e-001+1.01643954e-019j  1.92062780e+000-6.43914447e-018j
   1.29479626e-017-2.75730704e-017j ... -1.50033577e-034+1.08411309e-050j
  -6.80247213e-035-2.75529829e-050j -1.11210758e-065-1.34086745e-081j]
 [-3.01265785e-001-1.70084216e-018j  6.71215787e-018-4.06575815e-020j
   2.20722606e+000-2.76046408e-017j ... -1.04900591e-034-3.17165956e-050j
  -4.59887284e-035-1.21726208e-050j  3.81455601e-067+7.72195917e-082j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -8.97570715e-037-1.25051445e-052j
  -2.55827609e-036+2.83562051e-052j ...  4.39324370e+000+2.36085023e-017j
  -8.90768978e-018+1.11734618e-017j -1.08079631e+000-2.60208521e-018j]
 [ 3.48028785e-098+7.47416528e-114j -4.06537005e-037-6.85128581e-054j
  -1.13390332e-036+9.43720634e-053j ...  3.45960968e-017-4.840623

In [12]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





63
[[-2.55749532e+00+0.00000000e+00j  7.57191364e-01-2.22470189e-17j
   1.22541752e+00-2.30281365e-18j ... -3.21590966e-34+1.46076551e-51j
  -1.45764735e-34+2.17736544e-52j -8.82504401e-66+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -2.66991087e+00+0.00000000e+00j
   1.06043697e-02-2.64285345e-17j ... -4.57942278e-32-1.72752269e-48j
  -3.60228121e-32-1.93092473e-48j -1.17944197e-34+2.75238817e-51j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -2.60409121e+00+0.00000000e+00j ... -4.25922065e-32+6.50975709e-50j
  -2.54370632e-32-2.86638660e-48j -8.46586209e-35+2.04769520e-50j]
 ...
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ... -3.80769042e+00+0.00000000e+00j
   1.26784168e+00+3.39312320e-17j  2.53984874e+00-2.32099252e-18j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  -2.91356729e+00+0.00000000e+00j  2.9135672

In [13]:
print(diag_r)

[np.complex128(-2.5574953224588657+0j), np.complex128(-2.669910873113885+0j), np.complex128(-2.6040912089451442+0j), np.complex128(-2.365048792858441+0j), np.complex128(-2.2821930601873763+0j), np.complex128(-2.3079810921090744+0j), np.complex128(-2.490112192101455+0j), np.complex128(-2.8017822595967257+0j), np.complex128(-2.7832724555577837+0j), np.complex128(-2.484902268984881+0j), np.complex128(-2.700164231946677+0j), np.complex128(-2.599401536750435+0j), np.complex128(-2.4659539901474603+0j), np.complex128(-2.6834458774546985+0j), np.complex128(-2.607442246028976+0j), np.complex128(-2.702973079203711+0j), np.complex128(-2.6067713715936605+0j), np.complex128(-2.740256332155743+0j), np.complex128(-2.5000529908489773+0j), np.complex128(-2.4545526068199788+0j), np.complex128(-2.491708812818465+0j), np.complex128(-2.553747598207202+0j), np.complex128(-2.9133795556436133+0j), np.complex128(-3.145444376238775+0j), np.complex128(-3.1058219152533755+0j), np.complex128(-3.068546429689175+0j)

In [14]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

64
(64, 64)
[[ 1.26004965e+000+0.00000000e+000j -8.70313963e-001+0.00000000e+000j
  -1.24789653e+000+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.31309218e-125+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-2.10280741e-001+1.01643954e-019j  1.92062780e+000-6.43914447e-018j
   1.29479626e-017-2.75730704e-017j ... -1.50033577e-034+1.08411309e-050j
  -6.80247213e-035-2.75529829e-050j -1.11210758e-065-1.34086745e-081j]
 [-3.01265785e-001-1.70084216e-018j  6.71215787e-018-4.06575815e-020j
   2.20722606e+000-2.76046408e-017j ... -1.04900591e-034-3.17165956e-050j
  -4.59887284e-035-1.21726208e-050j  3.81455601e-067+7.72195917e-082j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -8.97570715e-037-1.25051445e-052j
  -2.55827609e-036+2.83562051e-052j ...  4.39324370e+000+2.36085023e-017j
  -8.90768978e-018+1.11734618e-017j -1.08079631e+000-2.60208521e-018j]
 [ 3.48028785e-098+7.47416528e-114j -4.06537005e-037-6.85128581e-054j
  -1.13390332e-036+9.43720634e-053j ...  3.45960968e-

In [15]:
print(A.shape)

(64, 64)


In [16]:
print(np.linalg.matrix_rank(A))
print(A.shape)

63
(64, 64)


In [17]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[2.74241552e-01+1.13043002e-17j 6.62608206e-02+2.75486861e-18j
 6.62070894e-02+3.61662144e-18j 6.60605161e-02+3.42221495e-18j
 6.58608173e-02+6.61066687e-19j 6.56617229e-02+3.02358033e-18j
 6.55163576e-02+2.42436134e-18j 1.60095957e-02+2.50542970e-19j
 1.59966137e-02+1.16415556e-19j 1.59818596e-02+1.23231908e-18j
 1.59611991e-02-4.93532592e-20j 1.59521946e-02-4.27667335e-19j
 1.59257074e-02-3.98864923e-19j 1.59129489e-02-2.28482008e-19j
 1.58930159e-02-4.18180851e-19j 1.58648447e-02+6.44367947e-19j
 1.58422530e-02+1.26457555e-19j 1.58297222e-02-5.84226193e-19j
 1.58051304e-02-4.50587439e-19j 1.57916021e-02+4.66207386e-19j
 1.57527087e-02-2.17436982e-19j 1.56974324e-02+1.93568335e-20j
 3.86815550e-03-2.51157138e-19j 3.86501885e-03-3.66264387e-19j
 3.86145411e-03-4.73478223e-19j 3.85727706e-03-5.86862027e-19j
 3.85646214e-03-2.66294831e-19j 3.85428651e-03-6.50079985e-19j
 3.84788679e-03-3.78641122e-19j 3.84480418e-03-3.93092908e-19j
 3.83998807e-03-5.13256491e-19j 3.83652314e-03-3.319219

In [18]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[-2.77555756e-17+0.00000000e+00j -1.38777878e-17-7.70371978e-34j
  2.35922393e-16-6.54816181e-33j  2.84457419e-34-3.85185989e-33j
 -2.08166817e-17-5.77778983e-34j -4.16333634e-17+7.70371978e-34j
 -6.93889390e-18+1.54074396e-33j -1.38777878e-17+9.62964972e-35j
 -3.81639165e-17-5.77778983e-34j -9.02056208e-17-3.85185989e-34j
 -2.77555756e-17+6.74075481e-34j -6.93889390e-18+9.62964972e-34j
 -1.04083409e-17+5.77778983e-34j -3.12250226e-17+1.92592994e-34j
 -3.46944695e-18+5.77778983e-34j  2.08166817e-17+5.77778983e-34j
 -4.51028104e-17+5.77778983e-34j  6.93889390e-18-1.92592994e-34j
  1.04083409e-17+1.92592994e-34j  6.39833261e-36+1.92592994e-33j
 -1.73472348e-17+0.00000000e+00j -1.73472348e-17+6.74075481e-34j
  5.20417043e-18-9.62964972e-35j  1.73472348e-18-6.74075481e-34j
  6.93889390e-18+1.92592994e-34j -1.73472348e-18-5.29630735e-34j
  1.73472348e-18+9.62964972e-35j  2.60208521e-18+3.85185989e-34j
 -8.67361738e-19+0.00000000e+00j  2.60208521e-18+1.92592994e-34j
  2.60208521e-18+1.685188

In [19]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j -5.75754671e-84-0.j -7.20269094e-81-0.j
  9.97707536e-26+0.j  0.00000000e+00+0.j -9.97750790e-26+0.j
 -4.68927578e-18+0.j  7.44164867e-30+0.j  6.93446618e-39+0.j
  4.68927558e-18+0.j  2.00350271e-22+0.j  1.62645715e-28+0.j
 -2.76943731e-22+0.j  1.69750167e-16+0.j -1.69218316e-16+0.j
 -3.47076771e-19+0.j  2.16692542e-36+0.j  4.68887508e-18+0.j
  3.61763387e-22+0.j -1.69750168e-16+0.j -2.45816511e-22+0.j
  1.45324297e-17+0.j  7.48129298e-17+0.j  1.36203428e-17+0.j
  4.68870187e-18+0.j -1.46662039e-16+0.j  3.47937418e-17+0.j
  3.84671301e-17+0.j -1.38911464e-16+0.j -1.73142083e-16+0.j
 -1.15759670e-16+0.j -4.08248290e-01+0.j  1.00408924e-33+0.j
 -3.30354054e-22+0.j -4.68835463e-18+0.j  1.40153459e-16+0.j
 -4.68902623e-18+0.j  5.20265203e-17+0.j  2.06555726e-17+0.j
 -1.76440484e-16+0.j -1.19785237e-22+0.j -4.79489729e-17+0.j
  6.25997904e-17+0.j  1.98793246e-16+0.j  2.21017512e-16+0.j
  2.22708655e-16+0.j -2.92101574e-16+0.j -4.08248290e-01+0.j
  2.00903823e-22+0.j -9.

In [20]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))

print(rho_comp)
print(np.trace(rho_comp))

[[ 5.45600317e-05+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.05966960e-03+0.j -9.82652642e-04+0.j ...
   1.60132166e-35+0.j  8.00140657e-36+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -9.82652642e-04+0.j  1.48042248e-03+0.j ...
   2.74710686e-35+0.j  1.37381146e-35+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  1.60132166e-35+0.j  2.74710686e-35+0.j ...
   4.00124002e-02+0.j -2.25915364e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  8.00140657e-36+0.j  1.37381146e-35+0.j ...
  -2.25915364e-02+0.j  4.06980339e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  2.74241552e-01+0.j]]
(1.0000000000000018+0j)


In [21]:
def L2_red(rho,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*C(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies) + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*D(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*C(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies) + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*D(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [22]:
L2_redfield = L2_red(rho_comp,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb)

In [23]:
print(L2_redfield)

[[-3.05753591e-03+0.00000000e+00j  9.54009910e-04-1.50507354e-06j
   3.66529238e-04+1.42769551e-06j ...  1.12036931e-25-6.96037398e-25j
   1.87015105e-25-2.59392848e-25j  0.00000000e+00+0.00000000e+00j]
 [ 9.54009910e-04+1.50507354e-06j  1.25370550e-03+0.00000000e+00j
  -3.96625302e-03+6.18729147e-07j ... -2.73550446e-21-4.12063653e-21j
  -6.08408461e-22-8.88334383e-22j -3.61425642e-30-2.02169746e-30j]
 [ 3.66529238e-04-1.42769551e-06j -3.96625302e-03-6.18729147e-07j
   3.07643046e-03+0.00000000e+00j ...  2.52692686e-21+4.50109522e-21j
   5.54982205e-22+9.83883803e-22j -1.61431332e-28-5.14012637e-30j]
 ...
 [ 1.12036931e-25+6.96037398e-25j -2.73550446e-21+4.12063653e-21j
   2.52692686e-21-4.50109522e-21j ... -8.46953179e-02+0.00000000e+00j
  -3.00580954e-02-1.32545004e-05j -4.97501553e-03+2.71944864e-06j]
 [ 1.87015105e-25+2.59392848e-25j -6.08408461e-22+8.88334383e-22j
   5.54982205e-22-9.83883803e-22j ... -3.00580954e-02+1.32545004e-05j
  -1.00592039e-01+0.00000000e+00j  3.72818225e-

In [24]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_6.mat',data_dict)


[[3.80777093e-04+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.03175783e-03+0.j 3.29634248e-06+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 3.29634248e-06+0.j 1.02846676e-03+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 ...
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  5.61523826e-02+0.j 1.79974202e-04+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  1.79974202e-04+0.j 5.63320690e-02+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 1.53616443e-01+0.j]]
